# Extract Image by Levels

## Load Image Information

In [219]:
import os
from scipy.io import loadmat

gt_dir = './widerface_evaluate/ground_truth'

hard_mat = loadmat(os.path.join(gt_dir, 'wider_hard_val.mat'))
medium_mat = loadmat(os.path.join(gt_dir, 'wider_medium_val.mat'))
easy_mat = loadmat(os.path.join(gt_dir, 'wider_easy_val.mat'))

hard_gt_list = hard_mat['gt_list']
medium_gt_list = medium_mat['gt_list']
easy_gt_list = easy_mat['gt_list']

In [220]:
gt_mat = loadmat(os.path.join(gt_dir, 'wider_face_val.mat'))
event_list = gt_mat['event_list']
file_list = gt_mat['file_list']

## Extract Image Paths by Levels

In [235]:
from collections import defaultdict
import tqdm

EASY, MEDIUM, HARD = 0, 1, 2
IMAGE_LEVEL = (EASY, MEDIUM, HARD)

setting_gts = [easy_gt_list, medium_gt_list, hard_gt_list]
img_paths = defaultdict(list)
for img_level in IMAGE_LEVEL:
    gt_list = setting_gts[img_level]
    for i in tqdm.tqdm(range(len(event_list))):
        img_dir = f'./data/widerface/val/images/{event_list[i][0][0]}'
        img_list = file_list[i][0].flatten()
        sub_gt_list = gt_list[i][0]
        count = 0
        for j, img_name in enumerate(img_list):
            if len(sub_gt_list[j][0]) != 0:
                count += 1
                img_paths[img_level].append(f'{img_dir}/{img_name[0]}.jpg')

# 이상하네 HARD에서 이미지 몇 개가 빠지는 군...
len(img_paths[EASY]), len(img_paths[MEDIUM]),len(img_paths[HARD])

100%|██████████| 61/61 [00:00<00:00, 6123.80it/s]


(2604, 3019, 3219)

In [248]:
temp_img_paths_for_hard = set(img_paths[HARD]) - set(img_paths[MEDIUM]) - set(img_paths[EASY])
temp_img_paths_for_medium = set(img_paths[MEDIUM]) - set(img_paths[EASY]) - set(img_paths[HARD])
img_paths[HARD] = temp_img_paths_for_hard
img_paths[MEDIUM] = temp_img_paths_for_medium
len(img_paths[EASY]), len(img_paths[MEDIUM]),len(img_paths[HARD])

(2604, 415, 200)

## Separate and Copy Images by Levels

In [247]:
import shutil

LEVELS = ('easy', 'medium', 'hard')
target_dir = './data/widerface/val/images_by_level'
if not os.path.isdir(target_dir):
    for level in LEVELS:
        os.makedirs(f'{target_dir}/{level}')

for img_level, ori_img_paths in img_paths.items():
    for ori_img_path in ori_img_paths:
        img_name = ori_img_path.split('/')[-1]
        target_img_path = f'{target_dir}/{LEVELS[img_level]}/{img_name}'
        shutil.copy(ori_img_path, target_img_path)